# **Differentially Private Deep Learning in 20 lines of code**

This is a step-by-step tutorial on how to train a simple PyTorch classification model on MNIST dataset using a differentially private - stochastic gradient descent optimizer in 20 lines of code using the PyTorch opacus library.  

Link to blogpost: https://blog.openmined.org/differentially-private-deep-learning-using-opacus-in-20-lines-of-code/

Link to library: https://github.com/pytorch/opacus

Author: Kritika Prakash, OpenMined


### **Step 1: Importing PyTorch and Opacus**

In [55]:
import torch
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

In [56]:
# !pip install -e .

## **Step 2: Loading MNIST data**

In [57]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../mnist',
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,)),]),),
                   batch_size=64,
                   shuffle=True,
                   num_workers=1,
                   pin_memory=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../mnist', 
                       train=False, 
                       transform=transforms.Compose([transforms.ToTensor(), 
                                                     transforms.Normalize((0.1307,), (0.3081,)),]),), 
                       batch_size=1024,
                       shuffle=True,
                       num_workers=1,
                       pin_memory=True)

## **Step 3: Creating a Neural Network Classification Model and Optimizer**

In [61]:
model = torch.nn.Sequential(
        torch.nn.Conv2d(1, 16, 8, 2, padding=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 1),
        torch.nn.Conv2d(16, 32, 4, 2),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 1),
        torch.nn.Flatten(),
        torch.nn.Linear(32 * 4 * 4, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

## **Step 4: Creating and Attaching a Differential Privacy Engine to the Optimizer**

In [62]:
privacy_engine = PrivacyEngine(
    model,
    batch_size=64,
    sample_size=60000,
    alphas=range(2,32),
    noise_multiplier=1.3,
    max_grad_norm=1.0,
)

privacy_engine.attach(optimizer)

## **Step 5: Creating a training function**

In [63]:
def train(model, train_loader, optimizer, epoch, device, delta):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    
    epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(delta)
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta}) for α = {best_alpha}"
    )

## **Step 6: Training the private model over multiple epochs**

In [64]:
for epoch in range(1, 11):
    train(model, train_loader, optimizer, epoch, device="cpu", delta=1e-5)

  0%|          | 0/938 [00:00<?, ?it/s]

Train Epoch: 1 	Loss: 1.264175 (ε = 0.55, δ = 1e-05) for α = 23


 53%|█████▎    | 498/938 [00:24<00:33, 13.03it/s]Exception ignored in: <function PerSampleGradientClipper.__del__ at 0x7f0de53fc0e0>
Traceback (most recent call last):
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 113, in __del__
    self.close()
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 119, in close
    if self.hooks_attached:  # do not close twice
AttributeError: 'PerSampleGradientClipper' object has no attribute 'hooks_attached'
Exception ignored in: <function PerSampleGradientClipper.__del__ at 0x7f0de53fc0e0>
Traceback (most recent call last):
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 113, in __del__
    self.close()
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 119, in close
    if self.hooks_attached:  # do not close twice
AttributeError: 'PerSampleGradientClipper' object has no attribute 'hooks_attached'
  0%|          | 0/938 [00:00<?, ?it/s]

Train Epoch: 2 	Loss: 0.581603 (ε = 0.57, δ = 1e-05) for α = 22


 94%|█████████▍| 882/938 [00:50<00:02, 23.56it/s]Traceback (most recent call last):
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 113, in __del__
    self.close()
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 119, in close
    if self.hooks_attached:  # do not close twice
AttributeError: 'PerSampleGradientClipper' object has no attribute 'hooks_attached'
Exception ignored in: <function PerSampleGradientClipper.__del__ at 0x7f0de53fc0e0>
Traceback (most recent call last):
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 113, in __del__
    self.close()
  File "/home/kritika/opacus/opacus/per_sample_gradient_clip.py", line 119, in close
    if self.hooks_attached:  # do not close twice
AttributeError: 'PerSampleGradientClipper' object has no attribute 'hooks_attached'
  0%|          | 0/938 [00:00<?, ?it/s]

...

Train Epoch: 8 	Loss: 0.558948 (ε = 0.63, δ = 1e-05) for α = 22


  0%|          | 0/938 [00:00<?, ?it/s]

Train Epoch: 9 	Loss: 0.597282 (ε = 0.64, δ = 1e-05) for α = 22


100%|██████████| 938/938 [00:38<00:00, 24.22it/s]

Train Epoch: 10 	Loss: 0.601695 (ε = 0.65, δ = 1e-05) for α = 22


## **Putting the code all together**

In [ ]:
# Step 1: Importing PyTorch and Opacus
import torch
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

# Step 2: Loading MNIST Data
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist', train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), 
               (0.3081,)),]),), batch_size=64, shuffle=True, num_workers=1, pin_memory=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist', train=False, 
              transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), 
              (0.3081,)),]),), batch_size=1024, shuffle=True, num_workers=1, pin_memory=True)

# Step 3: Creating a PyTorch Neural Network Classification Model and Optimizer
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3), torch.nn.ReLU(), torch.nn.MaxPool2d(2, 1),
        torch.nn.Conv2d(16, 32, 4, 2),  torch.nn.ReLU(), torch.nn.MaxPool2d(2, 1), torch.nn.Flatten(), 
        torch.nn.Linear(32 * 4 * 4, 32), torch.nn.ReLU(), torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

# Step 4: Attaching a Differential Privacy Engine to the Optimizer
privacy_engine = PrivacyEngine(model, batch_size=64, sample_size=60000, alphas=range(2,32), 
                               noise_multiplier=1.3, max_grad_norm=1.0,)

privacy_engine.attach(optimizer)

# Step 5: Training the private model over multiple epochs
def train(model, train_loader, optimizer, epoch, device, delta):
    
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(delta)
    
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta}) for α = {best_alpha}"
    )
    
for epoch in range(1, 11):
    train(model, train_loader, optimizer, epoch, device="cpu", delta=1e-5)